#### Comparing Sentinel C-Band SAR to Capella X-Band SAR

In [18]:
import json, numpy as np
from pyproj import Transformer
import ee, geemap, rasterio
import os

ee.Initialize(project='ee-krle4401')

# --- 1) Load Capella metadata ---
json_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/data/raw/scene_shanghai/CAPELLA_C11_SP_GEO_HH_20250320045730_20250320045802_extended.json"
with open(json_path, "r") as f:
    meta = json.load(f)

# Geotransform (UTM 51N, EPSG:32651)
gt = meta["collect"]["image"]["image_geometry"]["geotransform"]
rows = int(meta["collect"]["image"]["rows"])
cols = int(meta["collect"]["image"]["columns"])
x0, px_w, rot_x, y0, rot_y, px_h_neg = gt
px_h = abs(px_h_neg)

# Corners in UTM (no rotation for this product)
width_m  = px_w * cols
height_m = px_h * rows
corners_utm = np.array([
    [x0,             y0            ],  # TL
    [x0 + width_m,   y0            ],  # TR
    [x0 + width_m,   y0 - height_m ],  # BR
    [x0,             y0 - height_m ],  # BL
    [x0,             y0            ],  # close
], dtype=float)

# Reproject to lon/lat
transformer = Transformer.from_crs(32651, 4326, always_xy=True)
lons, lats = transformer.transform(corners_utm[:,0], corners_utm[:,1])
footprint_ll = [[float(lon), float(lat)] for lon, lat in zip(lons, lats)]
centroid_lon = float(np.mean(lons[:-1])); centroid_lat = float(np.mean(lats[:-1]))

# --- 2) Correct center time from JSON ---
# Prefer center_pixel.center_time; fall back to start/stop if needed
center_iso = (meta.get("collect", {})
                  .get("image", {})
                  .get("center_pixel", {})
                  .get("center_time"))
if center_iso is None:
    center_iso = meta["collect"]["start_timestamp"]  # safe fallback

capella_date = ee.Date(center_iso)
print("Capella center time:", center_iso)

# AOI polygon
aoi = ee.Geometry.Polygon([footprint_ll])

# --- 3) Find closest Sentinel scene (HH if available, else VV/VH) ---
window_days = 180
s1_hh = (ee.ImageCollection('COPERNICUS/S1_GRD')
         .filterBounds(aoi)
         .filterDate(capella_date.advance(-window_days,'day'),
                     capella_date.advance(window_days,'day'))
         .filter(ee.Filter.listContains('transmitterReceiverPolarisation','HH')))

def add_time_diff(img):
    return img.set('time_diff_days', img.date().difference(capella_date, 'day').abs())

s1_hh_sorted = s1_hh.map(add_time_diff).sort('time_diff_days')

if s1_hh_sorted.size().getInfo() > 0:
    s1_img  = ee.Image(s1_hh_sorted.first()); s1_band = 'HH'
    s1_vis  = {'bands':[s1_band], 'min':-25, 'max':0}
    print("Closest Sentinel‑1 HH:", s1_img.date().format().getInfo())
else:
    s1_iw = (ee.ImageCollection('COPERNICUS/S1_GRD')
             .filterBounds(aoi)
             .filterDate(capella_date.advance(-window_days,'day'),
                         capella_date.advance(window_days,'day'))
             .filter(ee.Filter.eq('instrumentMode','IW'))
             .sort('system:time_start'))
    s1_img = ee.Image(s1_iw.first())
    bands  = s1_img.bandNames().getInfo()
    s1_band = 'VV' if 'VV' in bands else 'VH'
    s1_vis  = {'bands':[s1_band],
               'min': -22 if s1_band=='VV' else -27,
               'max':   0 if s1_band=='VV' else  -5}
    print("Closest Sentinel‑1 (IW):", s1_img.date().format().getInfo(), "| Band:", s1_band)

# --- 4) Map & local Capella overlay ---
Map = geemap.Map(center=[centroid_lat, centroid_lon], zoom=13)
Map.addLayer(s1_img.select(s1_band), s1_vis, f"S1 {s1_band} (closest)")
Map.addLayer(aoi, {'color':'red'}, "Capella footprint (exact)")

capella_path = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs/shanghai_hh_db.tif"  # adjust if needed
with rasterio.open(capella_path) as src:
    arr = src.read(1, masked=True).compressed()
    p2, p98 = np.percentile(arr, [2, 98])

Map.add_raster(capella_path, layer_name="Capella HH (local dB)",
               vmin=float(p2), vmax=float(p98), opacity=0.65)

Map.split_map(left_layer=f"S1 {s1_band} (closest)", right_layer="Capella HH (local dB)")
project_dir = "/Users/kitlewers/Capella_SAR_Fun/capella-sar-seg/outputs"
os.makedirs(project_dir, exist_ok=True)
out_html = os.path.join(project_dir, "sentinel_capella_compare.html")
Map.to_html(out_html, title="Capella vs Sentinel (aligned)", width="100%", height="820px")
print("Saved: sentinel_capella_compare.html")


Capella center time: 2025-03-20T04:57:46.562000Z
Closest Sentinel‑1 HH: 2025-04-11T09:45:46


KeyboardInterrupt: 

Even just looking between the two, the difference is staggering. It is worth reminding you that these are different bands. 

<p float="left">
  <img src="https://raw.githubusercontent.com/kllewers/Capella_Space_Dabbling/main/Sentinel_data.png" width="49%" />
  <img src="https://raw.githubusercontent.com/kllewers/Capella_Space_Dabbling/main/Sentinel_Capella_Data.png" width="49%" />
</p>


In [ ]:
import ee
ee.Initialize(project='ee-krle4401')

# 0) Inputs
aoi = ee.Geometry.Polygon([footprint_ll])   # from your earlier code (exact Capella footprint)
s1   = s1_img.select([s1_band])             # from your earlier code (closest S1, 'HH' or VV/VH)
cap  = ee.Image("users/krle4401/capella_hh_20250320_db").select([0]).rename('CAP')  # 'b1' or 0, adjust if needed

# 1) Build masks that exclude nodata for each image
#    - Sentinel-1 GRD already has a proper mask outside footprint; keep >-60 dB sanity.
s1_valid  = s1.updateMask(s1).gt(-60)

#    - Capella: many products use 0 as nodata; exclude zeros and non-finite; keep >-60 dB sanity.
cap_valid = (cap.updateMask(cap.neq(0))
                .updateMask(cap.isFinite())
                .gt(-60))

# 2) Intersection mask: valid in both AND inside AOI
roi_mask = ee.Image.constant(1).clip(aoi).selfMask()
both_mask = s1_valid.mask().And(cap_valid.mask()).And(roi_mask.mask())

# 3) Apply the mask to a 2‑band stack (S1, Capella)
stack = (s1.rename('S1')
         .addBands(cap.rename('CAP'))
         .updateMask(both_mask))

# 4A) Quick visual check (replace vis ranges if your CAP is not dB)
vis = {'min': -25, 'max': 0}
Map = geemap.Map(center=[centroid_lat, centroid_lon], zoom=13)
Map.addLayer(stack.select('S1'), vis, 'S1 (masked to overlap)')
Map.addLayer(stack.select('CAP'), vis, 'Capella (masked to overlap)')
Map.addLayer(aoi, {'color':'red'}, 'AOI')
Map

# 4B) Optional: sample only overlapping valid pixels (e.g., 10k points)
samples = (stack.sample(
    region=aoi,
    scale=10,               # pick analysis scale; or use CAP pixel size (~0.2 m) if you’ve resampled
    numPixels=10000,
    geometries=False
))
print('Sample count:', samples.size().getInfo())

# 4C) Optional: export the masked 2‑band stack for local analysis
task = ee.batch.Export.image.toDrive(
    image=stack,
    description='S1_CAP_overlap_only',
    region=aoi,
    scale=10,               # choose common analysis scale
    maxPixels=1e13
)
task.start()


AttributeError: 'Image' object has no attribute 'isFinite'

#### Only look at overlapping pixels in the AOI with that have Data Values